In [8]:
import os
import streamlit as st
import pickle
import time
import langchain
import langchain_community
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings


from secret_key import API_KEY
from langchain_google_genai import ChatGoogleGenerativeAI


In [2]:
os.environ['GEMINI_API_KEY'] = API_KEY

# Intialize LLM 
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.6,
    google_api_key=os.environ.get("GEMINI_API_KEY")
)

In [3]:
# loadind the urls
loaders = UnstructuredURLLoader( urls=   [
    "https://www.moneycontrol.com/technology/nvidia-to-have-final-say-on-strong-earnings-season-for-big-tech-article-13055545.html",
    "https://www.moneycontrol.com/technology/anthropic-makes-web-search-free-for-all-claude-ai-users-begins-voice-beta-testing-article-13054268.html"
])
data = loaders.load()
print(data[0].page_content)
print(data[1].page_content)  

English

Hindi

Gujarati

Specials

Hello, Login

Hello, Login

Log-inor Sign-Up

My Account

My Profile

My Portfolio

My Watchlist

My Alerts

My Messages

Price Alerts

My Profile

My PRO

My Portfolio

My Watchlist

My Alerts

My Messages

Price Alerts

Logout

Loans up to ₹50 LAKHS

Fixed Deposits

Credit CardsLifetime Free

Credit Score

Chat with Us

Download App

Follow us on:

Network 18

Go Ad-Free

My Alerts

>->MC_ENG_DESKTOP/MC_ENG_NEWS/MC_ENG_NEWS_AS/MC_ENG_ROS_NWS_AS_ATF_728

Moneycontrol

Go PRO NowPRO

Moneycontrol PRO

Advertisement

Remove Ad

Business

Markets

Stocks

Economy

Companies

Trends

IPO

Opinion

EV Special

Offer Subscription Products

profile

Ambareesh Baliga

profile

CK Narayan

profile

T Gnanasekar

profile

Power Your Trade

profile

Currencies with Mecklai Financial

profile

Stock Reports

Options Trading WebinarMay 31

HomeTechnologyNvidia to have final say on strong earnings season for Big Tech

Trending Topics

Galaxy Z Fold 7Free Fire MAX

In [4]:
# splitting the data into chunks'
rec_splitter = RecursiveCharacterTextSplitter(
    #separators=['\n\n', '\n', ''],
    chunk_size=1000,
    chunk_overlap=200,)
docs = rec_splitter.split_documents(data)
len(docs)

20

In [5]:
docs[2]

Document(metadata={'source': 'https://www.moneycontrol.com/technology/nvidia-to-have-final-say-on-strong-earnings-season-for-big-tech-article-13055545.html'}, page_content='Nvidia Corp. faces the final test of an earnings season-driven rally that has sent its shares up more than 40% from an April low.\n\nThe world’s most valuable chipmaker reports Wednesday after market close — the last of the Big Tech cohort to do so. Results from Microsoft Corp., Meta Platforms Inc. and others showed that outlooks remain mostly intact despite uncertainty caused by President Donald Trump’s shifting tariff plans.\n\nStory continues below Advertisement\n\nRemove Ad\n\nThe reports, along with cooling trade tensions, have helped fuel a rebound in technology stocks that were at the heart of last month’s S&P 500 rout.\n\n“Tech results have generally been positive, and in some cases, exceptional,” said Tim Ghriskey, senior portfolio strategist at Ingalls & Snyder. “I think continued strength in Nvidia will s

In [9]:
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=os.environ.get("GEMINI_API_KEY")
)
vectorindex_gemini = FAISS.from_documents(docs, embeddings)

### To store the vector index on local machine

To store the vector index on the local machine, use the following code snippet:


file_path="file_path/vector_index.pkl"

 with open(file_path, "wb) as f:

      pickle.dump(vectorindex_gemini, f)
   

To load the saved vector store use this code snippet

if os.path.exist(file_path):
      with open(file_path, "rb") as f:
         vectorIndex = pickle.load(f)

In [11]:
file_path = "C:/Users/aumpa/OneDrive/Documents/GitHub/LangChain/LangChainTut/vector_index"

# Save the vector index using FAISS's save_local method
vectorindex_gemini.save_local(file_path)

In [12]:
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, 
                            retriever=vectorindex_gemini.as_retriever())

In [13]:
chain

RetrievalQAWithSourcesChain(verbose=False, combine_documents_chain=MapReduceDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Use the following portion of a long document to see if any of the text is relevant to answer the question. \nReturn any relevant text verbatim.\n{context}\nQuestion: {question}\nRelevant text, if any:'), llm=ChatGoogleGenerativeAI(model='models/gemini-2.0-flash', google_api_key=SecretStr('**********'), temperature=0.6, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x0000019FC0FF9100>, default_metadata=(), model_kwargs={}), output_parser=StrOutputParser(), llm_kwargs={}), reduce_documents_chain=ReduceDocumentsChain(verbose=False, combine_documents_chain=StuffDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'su

In [14]:
query = "Which new models were launched by anthropic recently?"
langchain.debug = True
chain({"question": query}, return_only_outputs=True)

C:\Users\aumpa\AppData\Local\Temp\ipykernel_18268\2852731678.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain({"question": query}, return_only_outputs=True)


[chain/start] [chain:RetrievalQAWithSourcesChain] Entering Chain run with input:
{
  "question": "Which new models were launched by anthropic recently?"
}
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "The update comes as part of Anthropic’s broader push to make Claude more dynamic and user-friendly. The company has also begun beta testing a new voice mode in its mobile apps, according to a report by Engadget.\n\nStory continues below Advertisement\n\nRemove Ad\n\nUnlike the existing dictation tool, voice mode allows for fluid back-and-forth conversations with Claude. Users can choose from five distinct voice options, and Claude will provide transcripts and summaries post-conversation. The voice feature is currently being tested with Sonnet 

{'answer': 'Anthropic recently launched two new models: Opus 4 and Sonnet 4.\n',
 'sources': 'https://www.moneycontrol.com/technology/anthropic-makes-web-search-free-for-all-claude-ai-users-begins-voice-beta-testing-article-13054268.html'}